In [48]:
from tqdm import tqdm
import feedparser
import json
import os
import feedparser
from bs4 import BeautifulSoup
from tqdm import tqdm
import html
import re

def html_to_text(s):
    if not s:
        return ""
    s = html.unescape(s)
    soup = BeautifulSoup(s, "html.parser")
    text = soup.get_text(" ", strip=True)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def collect_from_rss_feeds(rss_feeds, max_docs=200, doc_prefix="en"):
    docs = []
    seen_urls = set()
    doc_i = 0

    for rss_url in tqdm(rss_feeds, desc="Processing RSS feeds"):
        feed = feedparser.parse(rss_url)

        for entry in feed.entries:
            url = getattr(entry, "link", "").strip()
            if not url or url in seen_urls:
                continue

            title = getattr(entry, "title", "").strip()
            summary_raw = getattr(entry, "summary", "").strip()
            summary_text = html_to_text(summary_raw)

            date = getattr(entry, "published", "")

            seen_urls.add(url)

            docs.append({
                "doc_id": f"{doc_prefix}_{doc_i:06d}",
                "title": html_to_text(title),
                "body": summary_text,
                "url": url,
                "date": date,
                "language": doc_prefix,
                "token_count": len(summary_text.split())
            })

            doc_i += 1

            if len(docs) >= max_docs:
                return docs

    return docs

rss_feeds=[
    "https://www.dhakatribune.com/feed/business"
     
]


saving_path = r"C:\Users\RAZER\Documents\GitHub\Snake_ARC_2\CLIR_Project\data2"
file_path = os.path.join(saving_path, "document_en_big.json")

if os.path.exists(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        existing_docs = json.load(f)
else:
    existing_docs = []

existing_urls = {doc["url"] for doc in existing_docs}

new_docs = collect_from_rss_feeds(rss_feeds)
new_docs = [doc for doc in new_docs if doc["url"] not in existing_urls]

start_id = len(existing_docs)
for i, doc in enumerate(new_docs):
    doc["doc_id"] = f"en_{start_id + i:06d}"

all_docs = existing_docs + new_docs

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(all_docs, f, ensure_ascii=False, indent=2)

print(f"Added {len(new_docs)} new documents. Total: {len(all_docs)}")


Processing RSS feeds: 100%|██████████| 1/1 [00:01<00:00,  1.37s/it]

Added 100 new documents. Total: 100


In [1]:
import json

with open(r"C:\Users\RAZER\Documents\GitHub\Snake_ARC_2\CLIR_Project\data2\document_en_big.json", "r", encoding="utf-8") as f:
    docs_en = json.load(f)

print("Before filtering:", len(docs_en))




Before filtering: 190


In [31]:
from tqdm import tqdm
import feedparser
import json
import os

import requests # Make sure to import this

from bs4 import BeautifulSoup
import requests

def collect_from_rss_feeds(rss_feeds, max_docs=200):
    docs = []
    seen_urls = set()
    doc_i = 0
    headers = {'User-Agent': 'Mozilla/5.0'}

    for rss_url in tqdm(rss_feeds, desc="Processing RSS feeds"):
        try:
            response = requests.get(rss_url, headers=headers, timeout=15)
            feed = feedparser.parse(response.content)
            
            for entry in feed.entries:
                url = getattr(entry, "link", "").strip()
                if not url or url in seen_urls:
                    continue
                
                title = getattr(entry, "title", "").strip()
                raw_body = getattr(entry, "summary", "").strip()
                
                # --- CLEANING STEP ---
                # This removes all the <div>, <p>, etc., and returns clean text
                soup = BeautifulSoup(raw_body, "html.parser")
                clean_body = soup.get_text(separator=" ", strip=True)
                
                date = getattr(entry, "published", "")
                
                seen_urls.add(url)
                docs.append({
                    "doc_id": f"en_{doc_i:06d}",
                    "title": title,
                    "body": clean_body, # Cleaned text
                    "url": url,
                    "date": date,
                    "language": "en",
                    "token_count": len(clean_body.split())
                })
                doc_i += 1
                if len(docs) >= max_docs:
                    return docs
        except Exception as e:
            print(f"Error parsing {rss_url}: {e}")
            
    return docs

rss_feeds = [
    # --- ALREADY WORKING FOR YOU ---
    "https://www.tbsnews.net/rss.xml"
]



# The rest of your code remains the same...



saving_path = r"C:\Users\RAZER\Documents\GitHub\Snake_ARC_2\CLIR_Project\data2"
file_path = os.path.join(saving_path, "document_en.json")

if os.path.exists(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        existing_docs = json.load(f)
else:
    existing_docs = []

existing_urls = {doc["url"] for doc in existing_docs}

new_docs = collect_from_rss_feeds(rss_feeds)
new_docs = [doc for doc in new_docs if doc["url"] not in existing_urls]

start_id = len(existing_docs)
for i, doc in enumerate(new_docs):
    doc["doc_id"] = f"en_{start_id + i:06d}"

all_docs = existing_docs + new_docs

with open(file_path, "w", encoding="utf-8") as f:
    json.dump(all_docs, f, ensure_ascii=False, indent=2)

print(f"Added {len(new_docs)} new documents. Total: {len(all_docs)}")


Processing RSS feeds:   0%|          | 0/1 [00:00<?, ?it/s]

Processing RSS feeds: 100%|██████████| 1/1 [00:00<00:00,  1.01it/s]

Added 0 new documents. Total: 45


In [ ]:
rss_feeds = [
    # --- ALREADY WORKING FOR YOU ---
    "https://www.tbsnews.net/rss.xml",
    "https://en.prothomalo.com/feed",

    # --- NEW VERIFIED SOURCES (2025) ---
    # 1. The Daily Messenger (Very active and standard-compliant)
    "https://www.dailymessenger.net/rss.xml",
    
    # 2. Daily Sun (Section-specific feeds are currently more stable)
    "https://www.daily-sun.com/rss/national",
    "https://www.daily-sun.com/rss/business",
    
    # 3. Bangladesh Sangbad Sangstha (BSS - The National News Agency)
    "https://www.bssnews.net/rss/rss.xml",
    
    # 4. Financial Express (Focuses on Economy/Business)
    "https://thefinancialexpress.com.bd/rss/national",
    "https://thefinancialexpress.com.bd/rss/trade"
]



